In [2]:
import tensorflow as tf
import cv2 

print('Tensorflow Version: ', tf.__version__)
print('cv2 Version: ', cv2.__version__)
print('Available GPU Devices: ', tf.config.list_physical_devices('GPU'))

Tensorflow Version:  2.1.0
cv2 Version:  4.2.0
Available GPU Devices:  []


In [3]:
# Import other libraries

import shutil
import os
import pathlib
import zipfile
from glob import glob
import logging
import warnings

import numpy as np 
import pandas as pd
import csv
import imutils

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.style as style
import seaborn as sns
from tqdm import tqdm

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Dense, Dropout, Flatten, BatchNormalization, LeakyReLU
from tensorflow.keras.layers import Bidirectional, LSTM, Reshape
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, MaxPool2D, Dropout, Lambda, GlobalAveragePooling2D
from tensorflow.keras.callbacks import CSVLogger, TensorBoard, ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.backend as K

## Dataset Builder

In [4]:
if tf.config.list_physical_devices('GPU') = []:
    DATA_DIR = '/Users/nhanpham/CoderSchool/finalproject/data'
    RAW_DIR = os.path.join(DATA_DIR, 'raw')
    LOG_DIR = '/Users/nhanpham/CoderSchool/finalproject/src/log'
else:
    DATA_DIR = '/Users/nhanpham/CoderSchool/finalproject/data'
    RAW_DIR = os.path.join(DATA_DIR, 'raw')
    LOG_DIR = '/Users/nhanpham/CoderSchool/finalproject/src/log'

In [5]:
def get_data(dataset_name):
    dataset_dir = os.path.join(RAW_DIR, dataset_name)
    dataset = {"path":[], "label":[]}
    counter = 0
    for item in pathlib.Path(dataset_dir).glob('*/**/*'): 
        if item.is_file():
            dataset['path'].append(str(item))
            dataset['label'].append(str(item.parent).split('/')[-1])
            counter += 1
    
    return dataset, counter

In [298]:
hockey_vid, len_hockey = get_data('hockey')
print(len_hockey)

movie_vid, len_movie = get_data('movie')
print(len_movie)

vflow_vid, len_vflow = get_data('vflow')
print(len_vflow)

1000
201
248


## Extract figures from videos

In [300]:
def save_figures_from_video(dataset_name,
                            n_samples = 5, 
                            skip_frames = None,
                            fix_len = 16):

    print('Extracting frames from', dataset_name, 'videos :')
    counter = 0 
    img_paths, labels, video_name = [], [], []
    dataset = eval(dataset_name + '_vid')
    for video_file in tqdm(dataset['path'][:n_samples]):
        if dataset['label'][counter] == 'Violence':
            video_label = int('1')
        else:
            video_label = int('0')
        
        # PREPARE FOLDER TO STORE IMAGES
        video_frame_folder = os.path.join(DATA_DIR, 'img', dataset_name)
        video_filename = str(video_file).split('/')[-1]
        if not os.path.exists(video_frame_folder):
            os.makedirs(video_frame_folder)

        # EXTRACT FRAMES
        seq_len = 0
        vc = cv2.VideoCapture(video_file)
        if fix_len is not None:
            vid_len = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
            skip_frames = int(float(vid_len)/float(fix_len))
        vc.set(cv2.CAP_PROP_POS_MSEC, (seq_len * skip_frames))
        success, figure_ = vc.read()
        success = True

        while success:
            success, figure = vc.read()
            if seq_len % skip_frames == 0:
                if success:
                    figure_curr = figure
                    image_file = os.path.join(video_frame_folder, str(video_filename).split('.')[0] + "f_%d.jpg" % seq_len)
                    cv2.imwrite(image_file, figure_curr)
                    img_paths.append(image_file)
                    labels.append(video_label)
                    video_name.append(video_filename)
            seq_len += 1
        counter += 1
        
    print('Finished extracting frames.')
    return dict(img_paths = img_paths, labels = labels, video_name = video_name)

In [301]:
hockey_imgs = save_figures_from_video('hockey', n_samples = None, skip_frames = None, fix_len = 16)

Extracting frames from hockey videos :
Finished extracting frames.


In [7]:
path = os.path.join(DATA_DIR, 'img')
df = pd.DataFrame.from_dict(hockey_imgs)
df.to_csv(path + '/hockey.csv', index=False)

NameError: name 'hockey_imgs' is not defined

In [8]:
hockey_data = pd.read_csv(os.path.join(DATA_DIR, 'img') + '/hockey.csv')

In [9]:
np.array(hockey_data['img_paths'])

array(['/Users/nhanpham/CoderSchool/finalproject/data/img/hockey/no494_xvidf_0.jpg',
       '/Users/nhanpham/CoderSchool/finalproject/data/img/hockey/no494_xvidf_2.jpg',
       '/Users/nhanpham/CoderSchool/finalproject/data/img/hockey/no494_xvidf_4.jpg',
       ...,
       '/Users/nhanpham/CoderSchool/finalproject/data/img/hockey/fi466_xvidf_34.jpg',
       '/Users/nhanpham/CoderSchool/finalproject/data/img/hockey/fi466_xvidf_36.jpg',
       '/Users/nhanpham/CoderSchool/finalproject/data/img/hockey/fi466_xvidf_38.jpg'],
      dtype=object)

In [10]:
def split_data(dataframe):
    imgs = np.array(dataframe['img_paths'])
    labels = np.array(dataframe['labels'])
    X_train, X_val, y_train, y_val = train_test_split(imgs, labels, test_size = 0.2, random_state=101)
    return X_train, X_val, y_train, y_val

In [11]:
X_train, X_val, y_train, y_val = split_data(hockey_data)

In [12]:
num_train = len(X_train)
num_val = len(X_val)

print(num_train, num_val)

15980 3996


## Preprocessing 

In [13]:
def load_image_to_tensor(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize_with_crop_or_pad(image, 224, 224)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = image/255.

    return image

In [14]:
def load_and_preprocess_from_path_label(path, label):
    return load_image_to_tensor(path), label

In [18]:
BATCH_SIZE = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE

def create_dataset(filenames, labels, shuffle_buffer_size=1500, cache=False, training = True, augment = False):
    """Load and parse dataset.
       Args: filenames: list of image paths
             labels: numpy array of shape(BATCH_SIZE, N_LABELS)
             is_trainning: boolean to indicate training mode
             augment: boolean to indicate augment option. 
    """
    
    # Create a first dataset of file paths and labels
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    
    # Parse and preprocess observations in parallel
    dataset = dataset.map(load_and_preprocess_from_path_label, num_parallel_calls = AUTOTUNE)
    
    if training:
        if cache:
            if isinstance(cache, str):
                dataset = dataset.cache(cache)
            else: 
                dataset = dataset.cache()

        dataset = dataset.shuffle(buffer_size=shuffle_buffer_size)
        dataset = dataset.repeat()
        dataset = dataset.batch(BATCH_SIZE)
        
        if augment == True:
            dataset.map(augmentation, num_parallel_calls=AUTOTUNE)
        dataset = dataset.prefetch(buffer_size=AUTOTUNE)
        
    else: 
        dataset = dataset.batch(BATCH_SIZE)
        dataset = dataset.repeat()
        dataset = dataset.prefetch(buffer_size=AUTOTUNE)

    return dataset

In [19]:
train_ds = create_dataset(X_train, y_train)
val_ds = create_dataset(X_val, y_val)

In [20]:
train_ds

<PrefetchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int64)>

### Modeling

In [21]:
from tensorflow.keras.applications.vgg19 import VGG19

base_model = VGG19(weights='imagenet', input_shape = (224,224,3), include_top=False)

inputs = Input(shape=(224,224,3))
vgg = base_model(inputs)
cnn = Dense(512, activation = 'relu')(vgg)
shape = cnn.get_shape()
blstm = Reshape((shape[1], shape[2] * shape[3]))(cnn)
blstm = Bidirectional(LSTM(512, return_sequences=True, dropout = 0.2))(blstm)
blstm = Bidirectional(LSTM(512, return_sequences=True, dropout = 0.2))(blstm)
flat = Flatten()(blstm)
flat = BatchNormalization()(flat)

dense = Dense(512, activation = 'relu')(flat)
dropout = Dropout(0.5)(dense)
dense = Dense(128, activation = 'relu')(dropout)
dropout = Dropout(0.5)(dense)
dense = Dense(10, activation = 'relu')(dropout)

outputs = Dense(1, activation = 'sigmoid')(dense)

model = Model(inputs, outputs)

In [22]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
dense (Dense)                (None, 7, 7, 512)         262656    
_________________________________________________________________
reshape (Reshape)            (None, 7, 3584)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 7, 1024)           16781312  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 7, 1024)           6295552   
_________________________________________________________________
flatten (Flatten)            (None, 7168)              0     

In [23]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy')

In [24]:
def get_callbacks(checkpoint, monitor="val_loss", verbose=0):
        """Setup the list of callbacks for the model"""
        callbacks = [
            TensorBoard(
                log_dir=LOG_DIR,
                histogram_freq=10,
                profile_batch=0,
                write_graph=True,
                write_images=False,
                update_freq="epoch"),
            ModelCheckpoint(
                filepath=os.path.join(LOG_DIR, checkpoint),
                monitor=monitor,
                save_best_only=True,
                save_weights_only=True,
                verbose=verbose),
            EarlyStopping(
                monitor=monitor,
                min_delta=1e-8,
                patience=15,
                restore_best_weights=True,
                verbose=verbose),
            ReduceLROnPlateau(
                monitor=monitor,
                min_delta=1e-8,
                factor=0.2,
                patience=10,
                verbose=verbose)
        ]

        return callbacks

In [25]:
EPOCHS = 10

num_steps_train = tf.math.ceil(num_train/BATCH_SIZE)
num_steps_val = tf.math.ceil(num_val/BATCH_SIZE)

callbacks = get_callbacks("3001_weights.hdf5", monitor="val_loss", verbose=1)
# model.load_weights("/content/drive/My Drive/Colab Notebooks/week8/out/nhan_model_2201-2.hdf5")

history = model.fit(x=train_ds,
          epochs = EPOCHS,
          steps_per_epoch = num_steps_train,
          validation_data = val_ds, 
          validation_steps = num_steps_val, 
          verbose = 1,
          callbacks = callbacks)

Train for 500.0 steps, validate for 125.0 steps
Epoch 1/10
  6/500 [..............................] - ETA: 6:10:48 - loss: 0.9309

KeyboardInterrupt: 